<a href="https://colab.research.google.com/github/TranPhu1999/HCMUS-NLPS-AIChallenge/blob/Phu/HCMUS_NLPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tham khảo
- Paper: https://paperswithcode.com/paper/frozen-in-time-a-joint-video-and-image
- Towee repo: https://github.com/towhee-io/towhee
- Video-text retrival: https://codelabs.towhee.io/how-to-build-a-text-video-retrieval-engine/index#1
- Similar video: https://codelabs.towhee.io/build-a-reverse-video-search-engine-in-minutes/index#0

In [1]:
# !pip install -r requirements.txt

In [2]:
import towhee
import os
import glob
from PIL import Image
import pandas as pd
import numpy as np
import unicodedata

c:\Users\Admin\anaconda3\envs\env4ml\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Vietnamese Engslish Translate

In [3]:
dict_map = {
    "òa": "oà","Òa": "Oà","ÒA": "OÀ","óa": "oá","Óa": "Oá","ÓA": "OÁ","ỏa": "oả","Ỏa": "Oả","ỎA": "OẢ","õa": "oã",
    "Õa": "Oã","ÕA": "OÃ","ọa": "oạ","Ọa": "Oạ","ỌA": "OẠ","òe": "oè","Òe": "Oè","ÒE": "OÈ","óe": "oé","Óe": "Oé",
    "ÓE": "OÉ","ỏe": "oẻ","Ỏe": "Oẻ","ỎE": "OẺ","õe": "oẽ","Õe": "Oẽ","ÕE": "OẼ","ọe": "oẹ","Ọe": "Oẹ","ỌE": "OẸ",
    "ùy": "uỳ","Ùy": "Uỳ","ÙY": "UỲ","úy": "uý","Úy": "Uý","ÚY": "UÝ","ủy": "uỷ","Ủy": "Uỷ","ỦY": "UỶ","ũy": "uỹ",
    "Ũy": "Uỹ","ŨY": "UỸ","ụy": "uỵ","Ụy": "Uỵ","ỤY": "UỴ",
    }

def strip_accents(s):
       return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_vi2en = AutoTokenizer.from_pretrained("../AIChallenge_data/ViEnTranslate/", src_lang="vi_VN")
model_vi2en = AutoModelForSeq2SeqLM.from_pretrained("../AIChallenge_data/ViEnTranslate/")

def translate_vi2en(vi_text: str) -> str:
    for i, j in dict_map.items():
        vi_text = vi_text.replace(i, j)
    input_ids = tokenizer_vi2en(vi_text, return_tensors="pt").input_ids
    output_ids = model_vi2en.generate(
        input_ids,
        do_sample=True,
        top_k=100,
        top_p=0.8,
        decoder_start_token_id=tokenizer_vi2en.lang_code_to_id["en_XX"],
        num_return_sequences=1,
    )
    en_text = tokenizer_vi2en.batch_decode(output_ids, skip_special_tokens=True)
    en_text = " ".join(en_text)
    en_text = strip_accents(en_text)
    en_text = en_text.replace("\\","")
    return en_text

# vi_text = "Cô cho biết: trước giờ tôi không đến phòng tập công cộng, mà tập cùng giáo viên Yoga riêng hoặc tự tập ở nhà. Khi tập thể dục trong không gian riêng tư, tôi thoải mái dễ chịu hơn."
# print(translate_vi2en(vi_text))

# Towhee text-video inference

In [5]:
# !curl -L https://github.com/towhee-io/examples/releases/download/data/text_video_search.zip -O

In [6]:
# Nếu chạy không được lệnh này thì ra ngoài folder rồi tự unzip
# !unzip -q -o text_video_search.zip

In [7]:
# ls

In [8]:
# import pandas as pd
# import os

# raw_video_path = './test_1k_compress' # 1k test video path.
# test_csv_path = './MSRVTT_JSFUSION_test.csv' # 1k video caption csv.

# test_sample_csv_path = './MSRVTT_JSFUSION_test_sample.csv'

# sample_num = 1000 # you can change this sample_num to be smaller, so that this notebook will be faster.
# test_df = pd.read_csv(test_csv_path)
# print('length of all test set is {}'.format(len(test_df)))
# sample_df = test_df.sample(sample_num, random_state=42)

# sample_df['video_path'] = sample_df.apply(lambda x:os.path.join(raw_video_path, x['video_id']) + '.mp4', axis=1)

# sample_df.to_csv(test_sample_csv_path)
# print('random sample {} examples'.format(sample_num))

# df = pd.read_csv(test_sample_csv_path)

# df[['video_id', 'video_path', 'sentence']].head()

In [9]:
# from IPython import display
# from pathlib import Path
# import towhee
# from PIL import Image

# def display_gif(video_path_list, text_list):
#     html = ''
#     for video_path, text in zip(video_path_list, text_list):
#         html_line = '<img src="{}"> {} <br/>'.format(video_path, text)
#         html += html_line
#     return display.HTML(html)

    
# def convert_video2gif(video_path, output_gif_path, num_samples=16):
#     frames = (
#         towhee.glob(video_path)
#               .video_decode.ffmpeg(sample_type='uniform_temporal_subsample', args={'num_samples': num_samples})
#               .to_list()[0]
#     )
#     imgs = [Image.fromarray(frame) for frame in frames]
#     imgs[0].save(fp=output_gif_path, format='GIF', append_images=imgs[1:], save_all=True, loop=0)


# def display_gifs_from_video(video_path_list, text_list, tmpdirname = './tmp_gifs'):
#     Path(tmpdirname).mkdir(exist_ok=True)
#     gif_path_list = []
#     for video_path in video_path_list:
#         video_name = str(Path(video_path).name).split('.')[0]
#         gif_path = Path(tmpdirname) / (video_name + '.gif')
#         convert_video2gif(video_path, gif_path)
#         gif_path_list.append(gif_path)
#     return display_gif(gif_path_list, text_list)

In [10]:
# sample_show_df = sample_df[:3]
# video_path_list = sample_show_df['video_path'].to_list()
# text_list = sample_show_df['sentence'].to_list()
# tmpdirname = './tmp_gifs'
# display_gifs_from_video(video_path_list, text_list, tmpdirname=tmpdirname)

In [11]:
# from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

# connections.connect(host='127.0.0.1', port='19530')

# # def create_milvus_collection(collection_name, dim):
# #     if utility.has_collection(collection_name):
# #         utility.drop_collection(collection_name)
    
# #     fields = [
# #     FieldSchema(name='id', dtype=DataType.INT64, descrition='ids', is_primary=True, auto_id=False),
# #     FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=dim)
# #     ]
# #     schema = CollectionSchema(fields=fields, description='video retrieval')
# #     collection = Collection(name=collection_name, schema=schema)

# #     # create IVF_FLAT index for collection.
# #     index_params = {
# #         'metric_type':'L2', #IP
# #         'index_type':"IVF_FLAT",
# #         'params':{"nlist":2048}
# #     }
# #     collection.create_index(field_name="embedding", index_params=index_params)
# #     return collection

# # collection = create_milvus_collection('text_video_retrieval', 512)

In [12]:
# import os
# import towhee

# # # device = 'cuda:0'
# # # device = 'cpu'
# # # For the first time you run this line, 
# # # it will take some time 
# # # because towhee will download operator with weights on backend.
# # dc = (
# #     towhee.read_csv(test_sample_csv_path)
# #       .runas_op['video_id', 'id'](func=lambda x: int(x[-4:]))
# #       .video_decode.ffmpeg['video_path', 'frames'](sample_type='uniform_temporal_subsample', args={'num_samples': 12})
# #       .runas_op['frames', 'frames'](func=lambda x: [y for y in x])
# #       .video_text_embedding.clip4clip['frames', 'vec'](model_name='clip_vit_b32', modality='video')
# #       .to_milvus['id', 'vec'](collection=collection, batch=30)
# # )
# # print('Total number of inserted data is {}.'.format(collection.num_entities))

In [13]:
# # device = 'cuda:0'
# dc = (
#     towhee.read_csv(test_sample_csv_path).unstream()
#       .video_text_embedding.clip4clip['sentence','text_vec'](model_name='clip_vit_b32', modality='text')
#       .milvus_search['text_vec', 'top10_raw_res'](collection='text_video_retrieval', limit=10)
#       .runas_op['video_id', 'ground_truth'](func=lambda x : [int(x[-4:])])
#       .runas_op['top10_raw_res', 'top1'](func=lambda res: [x.id for i, x in enumerate(res) if i < 1])
#       .runas_op['top10_raw_res', 'top5'](func=lambda res: [x.id for i, x in enumerate(res) if i < 5])
#       .runas_op['top10_raw_res', 'top10'](func=lambda res: [x.id for i, x in enumerate(res) if i < 10])
# )

# dc.select['video_id', 'sentence', 'ground_truth', 'top10_raw_res', 'top1', 'top5', 'top10']().show()

In [14]:
# benchmark = (
#     dc.with_metrics(['mean_hit_ratio',])
#         .evaluate['ground_truth', 'top1'](name='recall_at_1')
#         .evaluate['ground_truth', 'top5'](name='recall_at_5')
#         .evaluate['ground_truth', 'top10'](name='recall_at_10')
#         .report()
# )

In [15]:
# import gradio
# import towhee.models.clip4clip

# device = 'cuda:0'
# collection='text_video_retrieval'
# show_num = 3
# with towhee.api() as api:
#     milvus_search_function = (
#          api.clip4clip(model_name='clip_vit_b32', modality='text', device=device)
#             .milvus_search(collection=collection, limit=show_num)
#             .runas_op(func=lambda res: [os.path.join(raw_video_path, 'video' + str(x.id) + '.mp4') for x in res])
#             .as_function()
#     )

# interface = gradio.Interface(milvus_search_function, 
#                              inputs=[gradio.Textbox()],
#                              outputs=[gradio.Video(format='mp4') for _ in range(show_num)]
#                             )

# interface.launch(inline=True, share=True)

# Towhee text-image inference

In [16]:
# img_path_df = pd.DataFrame(columns=["img_path","video_id","frame"])
# img_path_df

In [17]:
# for video in os.listdir("../AIChallenge_data/KeyFrames"):
#     fix_frame_df = pd.read_csv(os.path.join("../AIChallenge_data/Fix_KeyFrames/",video +".csv"),names=["img","frame"])
#     for i in range(len(fix_frame_df)):
#         img_path_df = img_path_df.append({"img_path":os.path.join("KeyFrames",video,fix_frame_df.iloc[i]["img"]),
#         "video_id":video,"frame":fix_frame_df.iloc[i]["frame"]},ignore_index=True)

In [18]:
# img_path_df.to_csv("key_frame_df.csv")

In [19]:
# a = "./train/brain_coral/n01917289_1783.JPEG"
# a.encode("utf-8")

In [20]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')

def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
    FieldSchema(name='id', dtype=DataType.INT64, descrition='ids', is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='text image search')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":512}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

In [21]:
# import towhee

# towhee.glob['path'](img_path[101:102]) \
#       .image_decode['path', 'img']() \
#       .image_text_embedding.clip['img', 'vec'](model_name='clip_vit_b32', modality='image') \
#       .tensor_normalize['vec','vec']() \
#       .select['img', 'vec']() \
#       .show()

In [22]:
# towhee.dc['text'](["A cat watching a computer screen while siting on a bench"]) \
#       .image_text_embedding.clip['text','vec'](model_name='clip_vit_b32', modality='text') \
#       .tensor_normalize['vec','vec']() \
#       .select['text', 'vec']() \
#       .show()

In [23]:
%cd ../AIChallenge_data/

d:\GitHub\AIChallenge_data


In [24]:
import pandas as pd

df_keyframes = pd.read_csv('key_frame_df.csv',index_col=0)

import cv2
from towhee._types.image import Image

id_img = df_keyframes.set_index('id')['img_path'].to_dict()
def read_images(results):
    imgs = []
    for re in results:
        # print(re)
        path = id_img[re.id]
        imgs.append(path)
    return imgs

In [25]:
# collection = create_milvus_collection('text_image_challenge_search', 512)
collection = create_milvus_collection('text_image_search', 512)
# collection = Collection('text_image_challenge_search')
dc = (
    towhee.read_csv('key_frame_df.csv')
      .runas_op['id', 'id'](func=lambda x: int(x))
      .set_parallel(4)
      .image_decode['img_path', 'img']()
      .image_text_embedding.clip['img', 'vec'](model_name='clip_vit_b32', modality='image')
      .tensor_normalize['vec','vec']()
      .to_milvus['id', 'vec'](collection='text_image_search', batch=100)
      
)
print('Total number of inserted data is {}.'.format(collection.num_entities))

In [26]:
# cd reverse_image_search

In [27]:
# query = "The video is a continuous sketch of a foreign criminal. The first picture depicts a criminal and two policemen sitting in the back. The second picture has 14 people, including one person wearing a black shirt."

In [28]:
queries = {}
with open("./query.txt","r") as f:
    lines = f.readlines()
    for line in lines:
        query_name = line.split("//")[0]
        query = line.split("//")[1]
        queries[query_name] = query

In [29]:

# for i in range(26,len(queries.keys())):
#   key = list(queries.keys())[i]
#   (
#       towhee.dc['text']([queries[key]])
#         .image_text_embedding.clip['text', 'vec'](model_name='clip_vit_b32', modality='text')
#         .tensor_normalize['vec','vec']()
#         .milvus_search['vec', 'result'](collection=collection, limit=100)
#         .runas_op['result', 'result_img'](func=read_images)
#         .select['result_img']()
#         .to_csv(key+".csv")      
#   )
#   print(i)


In [91]:
query_dir = "D:\GitHub\AIChallenge_data\query-pack-0\query-pack-0"
list_queries_file = os.listdir(query_dir)
vi_queries = []
for query in list_queries_file:
    with open(os.path.join(query_dir,query),"r",encoding='utf8') as f:
        read_line = f.readline()
        vi_queries.append(read_line)

vi_queries

['Bản tin về một trận đấu của đội tuyển bóng đá Việt Nam. Các cầu thủ Việt Nam đang ăn mừng sau một bàn thắng. Đây không phải là đội tuyển quốc gia mà là đội tuyển U19. Bàn thắng nâng tỷ số lên 2-0 xuất phát từ một pha đánh đầu cận thành. Đối thủ của U19 Việt Nam là U19 Brunei.',
 'Một hoạt động giải trí ưa thích của giới trẻ. Thả diều tại ngoại ô TPHCM. Trong bức hình có một chiếc diều khổng lồ hình con mực. Cùng ngày là phóng sự khởi công cầu Châu Đốc bắc qua song Hậu.',
 'Một cuộc đua xe đồng đội, gồm người lớn và trẻ em. Những chiếc xe này không có động cơ, thay vào đó chúng được đẩy bởi các vị phụ huynh. Tại vạch xuất phát, có 3 người mặc đồ hóa trang thành những chú ngựa vằn. Bản tin này được phát vào ngày mà hôm trước đó là ngày Quốc khánh của nước Thụy Sĩ.',
 'Một sở thú tại Trung Quốc với nhiều loài động vật khác nhau. Trong khung hình có 2 con hà mã. Một trong 2 đang uống nước. Ngoài hà mã, sở thú còn có voi và gấu trúc.',
 'Một bức hình của Tổng thống Mỹ Joe Biden. Tổng thốn

In [92]:
eng_queries = [translate_vi2en(x) for x in vi_queries]
eng_queries

['The news of a match of Vietnamese football team. Vietnamese players are celebrating after a goal. This is not the national team but the U19 team. The goal that lifted the score to 2-0 came from a close-fitting header. Opponents of Vietnam U19 are Brunei U19.',
 'A favorite recreational activity of young people. Flying kites in the suburbs of Ho Chi Minh City. In the picture there is a giant kite in the shape of a squid. On the same day was the launch of Chau Doc Bridge across the Hau parallel.',
 'A team car race, consisting of adults and children. These cars do not have engines, instead they are pushed by the parents. At the starting line, there are 3 people dressed up as zebras. This newscast was broadcast on the day that the day before was the Swiss National Day.',
 'A zoo in China with many different animals. In the frame there are 2 hippos. One of the 2 is drinking water. In addition to hippos, the zoo also has elephants and raccoons.',
 'A picture of U.S. President Joe Biden. T

In [93]:
(
      towhee.dc['text'](eng_queries)
        .image_text_embedding.clip['text', 'vec'](model_name='clip_vit_b32', modality='text')
        .tensor_normalize['vec','vec']()
        .milvus_search['vec', 'result'](collection=collection, limit=100)
        .runas_op['result', 'result_img'](func=read_images)
        .select['result_img']()
        .to_csv("test"+".csv")  
  )

In [94]:
from ast import literal_eval


df = pd.read_csv("./test.csv")
df.result_img =df.result_img.apply(literal_eval)

for i in range(len(list_queries_file)):
    with open("{}_result.txt".format(list_queries_file[i]),"w") as f:
        list_img = df["result_img"][i]
        for img in list_img:
            video_id = img.split("\\")[-2]
            false_frame = img.split("\\")[-1].split(".")[0]
            real_frame = df_keyframes[df_keyframes.false_frame == int(false_frame)][video_id==df_keyframes.video_id].frame.values[0]
            f.write(video_id + ".mp4, " + str(real_frame) + '\n')

In [58]:
# with towhee.api() as api:
#     milvus_search_function = (
#         api.image_text_embedding.clip(model_name='clip_vit_b32',modality='text')
#             .tensor_normalize()
#             .milvus_search(collection='text_image_search', limit=5)
#             .runas_op(func=lambda res: [id_img[x.id] for x in res])
#             .as_function()
#     )

# import gradio

# interface = gradio.Interface(milvus_search_function, 
#                              gradio.inputs.Textbox(lines=1),
#                              [gradio.outputs.Image(type="file", label=None) for _ in range(5)]
#                             )

# interface.launch(inline=True, share=True)

# Calculate score

In [105]:
import numpy as np

rank = [18,5,1,1,2,33,101,63,1,13]


accept_rank = [1,5,20,50,100]
score = []
for i in range(len(rank)):
    for j in range(len(accept_rank)):
        score.append(1 if rank[i] <= accept_rank[j] else 0)

total_score = np.sum(score) / 5
total_score


6.4